# **Projet Python 2A**
Charlotte de Romémont • Suzie Grondin • Michael Walkowiak

*Ce projet est réalisé dans le cadre du cours de Python de Lino Galiana pour l'année 2022-2023.*

L'objectif de ce projet est de mettre en avant les risques liés au stress hydrique, c'est à dire le manque d'eau, pour l'exploitation minière. L'extraction minière est indispensable pour l'économie mondiale et devrait fortement augmenter au cours des prochaines décennies, notamment car les technologies de la transition bas-carbone nécessitent une plus grande diversité de métaux que leurs alternatives utilisant des combustibles fossiles (voir par exemple https://www.ecologie.gouv.fr/sites/default/files/rapport_cgdd_01_mobilite_bas_carbone_fevrier2022.pdf pour les véhicules électriques). 

Peu d'études ont directement porté sur le risque hydrique dans le secteur minier, comme le notent les membres du projet GENERATE de l'IFP Énergies nouvelles (https://www.researchgate.net/publication/350514545_Les_pressions_sur_l'eau_face_ignoree_de_la_transition_energetique), malgré des conséquences déjà perceptibles du manque d'eau dans certaines régions minières d'importance mondiale (https://www.lesechos.fr/finance-marches/marches-financiers/les-mines-de-cuivre-au-chili-premieres-victimes-du-manque-deau-1340672).

Nous chercheront donc à évaluer quel pourcentage de la production de certains minéraux critiques se situe dans des zones à risque, tout en gardant à l'esprit que  

Réciproquement, il serait intéressant, avec plus de temps, de chercher à estimer l'impact de l'exploitation minière sur la consommation d'eau et donc sur l'augmentation prévisible du stress hydrique. Cette étude pourrait consister à isoler les zones minières.


# Données utilisées
Nous utiliserons plusieurs jeux de données en accès libre sur internet et dont la récupération et l'exploitation est plus ou moins complexe: 


• Un recensement mondial de zones d’exploitations minières, associé à l'article de Maus, V., Giljum, S., Gutschlhofer, J. et al. A global-scale data set of mining areas. Sci Data 7, 289 (2020). https://doi.org/10.1038/s41597-020-00624-w. De notre point de vue, l'important est de noter que les données mises à disposition par les auteurs consistent essentiellement en des polygones qui correspondent aux zones occupées par des exploitations minières. Les données sont en format geopackage (.gpkg), et le système de coordonées est WGS 84. Nénamoins, ces polygones ne comportent pas d'information sur le type d'exploitation minière, seulement la zone occupée. Le jeu de données comporte une information sur le pays qui contient les polygones miniers, mais nous avons pu constater des erreurs et 
    
• Un webscrapping du site https://www.mindat.org/. Ces données consistent en des points qui indiquent la situation des mines, et des informations fines sur la localité administrative, la province et le pays. Par rapport aux données issues de l'article de Mauss, nous avons donc une information plus fine sur la localisation et la possibilité d'identifier le type de minral extrait. Étant donné la grande diversité des minéraux qui peuvent nous intéresser, nous avons choisi de focaliser cette étape de webscrapping sur les mines de cuivre, mais la structure du site mindat.org nous permet d'être confiants quand à la possibilité de reproduire la méthode pour d'autres minéraux. 

• Les données de production mondiale et pays par pays pour de nombreux minéraux d'intérêt, sur le site https://www.world-mining-data.info/?World_Mining_Data___Data_Section qui est tenu par le ministère des finances de la république d'Autriche. Ces données sont au format xlsx et réparties dans plusieurs onglets.

• Des scénarios de stress hydrique, à partir des travaux du World Resources Institute dans le cadre du projet Aqueduct (https://www.wri.org/aqueduct). Ces données consistent en des polygones couvrant la surface de la terre et en un ensemble d'indicateurs qui permettent de construire de plus de 100 scénarios de stress hydrique prévu, pour les années 2020, 2030, 2040 et selon des hypothèses optimistes, neutres ou négatives. Les données géographiques sont au format Shapefile (SHP) et le système de coordonées est WGS 84.


## Webscraping

Le Webscraping est une technique d'extraction de contenu de site web. 
Nous avons eu recours à cette technique pour récolter les positions (longitude et latitude) des mines de cuivre sur le site "https://www.mindat.org/min-1209.html". Nous l'avons réalisé grâce au package selenium en conternant certains accès. Malheureusement, nous avons été repérés et le site à bloquer cette manière de récolter les informations avant que l'on ai enregistré le dataframe désiré. De ce fait, vous pouvez trouver notre code de Webscrapping dans notre dossier (nommé WebScrapping.ipynb) mais les données avec lesquelles nous travaillerons ont étées récoltées "à la main". 

# Obtention des données et prétraitement
## webscraping

## correction des pays des polygones miniers

## conversion des données de production minière (?)


# Visualisation des données géographiques par pays

# Évaluation du pourcentage de zones minières situées dans les différentes zones de stress hydrique au niveau mondial et par pays

## au niveau mondial

## par pays

# Tentative d'identification des types d'exploitations minières par pays et minéraux - le cas du Chili et du cuivre

# Conclusion et travaux complémentaires
Réciproquement, il serait intéressant, avec plus de temps, de chercher à estimer l'impact de l'exploitation minière sur la consommation d'eau et donc sur l'augmentation prévisible du stress hydrique. Cette étude pourrait consister à isoler les zones minières, et à comparer les données chiffrées des scénarios de stress hydrique et leurs évolution au cours du temps à des zones proches et similaires mais qui ne contiennent pas d'exploitations minières.